### API

In [1]:
api_key = "2f86db3b572b48ee8fe5a8937a035a97"

In [2]:
#uploading the local files for trancriptions
import requests
import sys

upload_endpoint = 'https://api.assemblyai.com/v2/upload'
filename = 'mic_recording.wav'
# print(str(sys.argv))
def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

headers = {'authorization': api_key}
response = requests.post(upload_endpoint,
                        headers=headers,
                        data=read_file(filename))

# print(response.json())
audio_url =response.json()['upload_url'] 

In [3]:
#transcription

transcript_endpoint = "https://api.assemblyai.com/v2/transcript"
json = { "audio_url": audio_url }

response = requests.post(transcript_endpoint, json=json, headers=headers)
#print(response.json())
job_id = response.json()['id']

In [5]:
#polling
#asking the assembly ai if the job is done or not
polling_endpoint = transcript_endpoint +'/'+job_id
polling_response = requests.get(polling_endpoint, headers=headers)


#asking assembly ai repetitively if the work is done
def get_transcription_result_url():
    while True:
        polling_response = requests.get(polling_endpoint, headers=headers)
        if polling_response.json()['status']=='completed':
            return polling_response
        elif polling_response.json()['status']=='error':
            return "error"
        
get_transcription_result_url()
data = polling_response.json()

##saving the transcript
text_filename = filename+ '.txt'
with open(text_filename,"w") as f:
    f.write(data['text'])
print('Transcription Saved!!')


Transcription Saved!!
